## Fine-tuning of Mixtral-8x7B-v0.1 on Med_QA med_qa_en_4options_bigbio_qa

Install and Load Required Libraries

In [1]:
! pip3 install -q -U transformers
! pip install -q -U datasets
! pip3 install -q -U peft
! pip install -q -U trl
! pip3 install -q -U auto-gptq
! pip3 install -q -U optimum
! pip3 install -q -U bitsandbytes

In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/kmb85/rds/hpc-work/huggingface'

In [2]:
import transformers
import torch
from datasets import load_dataset
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from trl import SFTTrainer

/home/kmb85/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kmb85/miniconda3/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Load Mixtral-8x7B-v0.1 and Tokenizer

In [4]:
model_name_or_path = "mistralai/Mixtral-8x7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4b_quant_type='nf4',
    torch_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    use_safetensors=True,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
    token=""
)

Loading checkpoint shards: 100%|██████████| 19/19 [07:19<00:00, 23.14s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True,
                                          token="")
tokenizer.pad_token=tokenizer.eos_token

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

### Load LoRA Adapter

In [7]:
config = LoraConfig(
    r=32,
    lora_alpha=16,
    bias="none",
    task_type="CASUAL_LM",
    target_modules=["q_proj", "v_proj"]
)

In [8]:
model=get_peft_model(model, config)

### Dataset preparation

In [3]:
dataset = load_dataset('bigbio/med_qa', 'med_qa_en_4options_bigbio_qa')

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question_id', 'document_id', 'question', 'type', 'choices', 'context', 'answer'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['id', 'question_id', 'document_id', 'question', 'type', 'choices', 'context', 'answer'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['id', 'question_id', 'document_id', 'question', 'type', 'choices', 'context', 'answer'],
        num_rows: 1272
    })
})

In [5]:
DEFAULT_PROMPT = "Below is a medical question and four choices for answer. Output the correct choice to answer the question."

def generate_train_prompt(data_point):
    question = data_point[ 'question']
    choices = data_point['choices']
    choices_str = ''
    answer = data_point['answer']
    for choice in choices:
        choices_str += choice + "\n"
    answer = data_point[ 'answer'][0]
    text = f'{DEFAULT_PROMPT}\n###Question:\n{question}\n###Choices:\n{choices_str}###Output:\n{answer}'
    return {'text': text, 'labels': answer}

In [6]:
train_dataset = dataset['train'].shuffle().map(generate_train_prompt)

Map: 100%|██████████| 10178/10178 [00:01<00:00, 7520.93 examples/s]


In [7]:
validation_dataset = dataset['validation'].shuffle().map(generate_train_prompt)

Map: 100%|██████████| 1272/1272 [00:00<00:00, 6783.40 examples/s]


In [8]:
def generate_test_prompt(data_point):
    question = data_point[ 'question']
    choices = data_point['choices']
    choices_str = ''
    for choice in choices:
        choices_str += choice + "\n"
    text = f'{DEFAULT_PROMPT}\n###Question:\n{question}\n###Choices:\n{choices_str}###Output:\n'
    return {'text':text}

In [9]:
test_dataset = dataset['test'].shuffle().map(generate_test_prompt)

Map: 100%|██████████| 1273/1273 [00:00<00:00, 4249.58 examples/s]


### Training

In [17]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=24,
    gradient_accumulation_steps=24,
    learning_rate=0.00004,
    bf16=True,
    num_train_epochs=16,
    save_strategy="epoch",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    output_dir='./experiments',
    remove_unused_columns=False,
    warmup_ratio=0.03,
    logging_strategy='steps',
    evaluation_strategy='steps',
    logging_steps=15,
    label_names=['labels'],
    eval_steps=15,
    group_by_length=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    args=training_args,
    tokenizer=tokenizer,
    dataset_text_field='text',
    peft_config=config,
    max_seq_length=4096
)

Map: 100%|██████████| 1272/1272 [00:00<00:00, 6107.66 examples/s]
/home/kmb85/miniconda3/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:282: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
model.config.use_cache = False
trainer.state.log_history = True
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kmb85 (cam_kiril). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/kmb85/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
15,1.554700,1.484966
30,1.478900,1.374104
45,1.325700,1.228122
60,1.220900,1.185369
75,1.181000,1.154134
90,1.148300,1.134472
105,1.137200,1.123099
120,1.122200,1.115658
135,1.114400,1.110974
150,1.107700,1.106784


Checkpoint destination directory ./experiments/checkpoint-17 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/kmb85/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory ./experiments/checkpoint-35 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/kmb85/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated

KeyboardInterrupt: 

### Save the fine-tuned model

In [19]:
model.save_pretrained(f'Mixtral-8x7B_med_qa_en_4options_bigbio_qa_batch_size_24')

### Evaluate the fine-tuned model

In [13]:
import requests

url = "http://127.0.0.1:5000/api/v1/generate"

In [14]:
test_dataset = test_dataset.shuffle(seed=42)

In [15]:
import textdistance
import tiktoken
import ast

def num_of_tokens_from_text(text):
    encoding=tiktoken.encoding_for_model(model_name='gpt-3.5-turbo')
    num_tokens=len(encoding.encode(text=text))
    return num_tokens


def similiary(str1, str2):
    return textdistance.hamming.normalized_similarity(str1, str2)

In [16]:
request = {
    'max_new_tokens': 200,
    'temperature': 0.1,
    'repetition_penalty': 1,
    'top_p': 0.7,
    'stopping_strings': ['\n', '###']
}
headers = {'Content-Type': 'application/json'}

In [17]:
total_correct = 0
num_samples = 1000

In [15]:
for i in range(num_samples):
    request['prompt'] = test_dataset[i]['text']
    response = requests.post(url, json=request)

    prediction = ast.literal_eval(response.text)["results"][0]['text'].lower()
    total_correct += similiary(test_dataset[i]['answer'][0].lower(), prediction)

In [16]:
correct_percentage = (total_correct / num_samples) * 100
print(f'Correctness percentage {correct_percentage}%')

Correctness percentage 55.01294239127286%


### Evaluate the RAG model

In [36]:
import requests

url_rag = "https://b1b6-131-111-184-110.ngrok-free.app/search"

payload = {
    "text": '',
    "number_documents": 5,
    'collection': 'med_qa_4options'
}

In [37]:
dataset['test'] = dataset['test'].shuffle(seed=42)

In [38]:
total_correct = 0
num_samples = 200

In [39]:
request = {
    'max_new_tokens': 200,
    'temperature': 0.1,
    'repetition_penalty': 1,
    'top_p': 0.7,
    'stopping_strings': ['\n', '###']
}
headers = {'Content-Type': 'application/json'}

In [40]:
def generate_rag_prompt(data_point):
    question = data_point['text']
    answer = data_point['answer']
    text = f'{question}{answer}\n'
    return text

In [41]:
DEFAULT_PROMPT = "Below are some medical questions wtih four choices and answers. Output the correct choice to answer the last question only based on the provided choices."


for i in range(num_samples):
    request['prompt'] = DEFAULT_PROMPT+'\n'

    payload['question'] = dataset['test'][i]['question']

    response_rag = requests.get(url_rag, json=payload)
    data_rag = response_rag.json()

    for record in data_rag:
        request['prompt'] += generate_rag_prompt(record)

    question = dataset['test'][i]['question']
    choices = dataset['test'][i]['choices']
    choices_str = ''
    for choice in choices:
        choices_str += choice + "\n"
    request['prompt'] += f'###Question:\n{question}\n###Choices:\n{choices_str}###Output:\n'
    response = requests.post(url, json=request)

    prediction = ast.literal_eval(response.text)["results"][0]['text'].lower()
    prediction = prediction.replace("'", '')
    prediction = prediction.replace('[', '')
    prediction = prediction.replace(']', '')

    total_correct += similiary(dataset['test'][i]['answer'][0].lower(), prediction)

In [42]:
correct_percentage = (total_correct / num_samples) * 100
print(f'Correctness percentage {correct_percentage}%')

Correctness percentage 67.8301710844763%
